In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import KFold

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import Sampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "nci"  # Dataset{gdsc or ccle}


args = Args()
res, drug_feature, exprs, mut, cna, null_mask, pos_num = load_data(args)

load nci


In [5]:
epochs = []
true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)
device = "cpu"

for train_index, test_index in kfold.split(np.arange(pos_num)):
    sampler = Sampler(res, train_index, test_index, null_mask)
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    epochs.append(epoch)
    true_datas = pd.concat([true_datas, translate_result(true_data)])
    predict_datas = pd.concat([predict_datas, translate_result(predict_data)])

epoch:   0 loss:0.822321 auc:0.5116
epoch:  20 loss:0.573831 auc:0.7039
epoch:  40 loss:0.465087 auc:0.7533
epoch:  60 loss:0.381535 auc:0.7982
epoch:  80 loss:0.342747 auc:0.8145
epoch: 100 loss:0.323400 auc:0.8228
epoch: 120 loss:0.312916 auc:0.8281
epoch: 140 loss:0.306689 auc:0.8312
epoch: 160 loss:0.302623 auc:0.8332
epoch: 180 loss:0.299801 auc:0.8346
epoch: 200 loss:0.297755 auc:0.8354
epoch: 220 loss:0.296228 auc:0.8360
epoch: 240 loss:0.295061 auc:0.8364
epoch: 260 loss:0.294156 auc:0.8366
epoch: 280 loss:0.293445 auc:0.8367
epoch: 300 loss:0.292881 auc:0.8367
epoch: 320 loss:0.292430 auc:0.8367
epoch: 340 loss:0.292063 auc:0.8368
epoch: 360 loss:0.291762 auc:0.8368
epoch: 380 loss:0.291511 auc:0.8368
epoch: 400 loss:0.291299 auc:0.8369
epoch: 420 loss:0.291120 auc:0.8369
epoch: 440 loss:0.290967 auc:0.8370
epoch: 460 loss:0.290834 auc:0.8371
epoch: 480 loss:0.290720 auc:0.8372
epoch: 500 loss:0.290620 auc:0.8372
epoch: 520 loss:0.290532 auc:0.8373
epoch: 540 loss:0.290455 auc

In [6]:
true_datas.reset_index(drop=True).to_csv("true_nci.csv")
predict_datas.reset_index(drop=True).to_csv("pred_nci.csv")